In [2]:
%matplotlib inline
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from builtins import open, bytes

import csv
import json

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split

import time

import math as math

import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

import string

##import Tensorflow namespaces
#import tensorflow as tf
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.preprocessing.sequence import pad_sequences
#from tensorflow.keras.layers import Embedding, Activation, Dense, Dropout, TimeDistributed, Dense, Activation, LSTM
#import tensorflow.keras.utils as kutils
#
#from tensorflow.keras.callbacks import EarlyStopping
#from tensorflow.keras.callbacks import ModelCheckpoint
#from tensorflow.keras.models import load_model
#
#from tensorflow.keras.callbacks import TensorBoard
#
## GPU
#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
########################
###### References ######
########################

# Some of the code is copied or derived from this notebook: https://colab.research.google.com/drive/15wa925dj7jvdvrz8_z3vU7btqAFQLVlG
# This was made by Rostyslav Neskorozhenyi (https://www.linkedin.com/in/slanj/)

In [4]:
###################
# Basic functions #
###################

In [5]:
def print_arr(arr):
    print("size = " + str(len(arr)))
    print(arr)

In [6]:
def print_header(text):
    width = len(text) + 6
    border = ""
    for i in range(width):
        border += "-"
    
    print(border)
    print("--", text, "--")
    print(border)

In [7]:
############
# Get data #
############

In [8]:
df_ep4 = pd.read_csv('Data/star_wars_epIV.txt', sep =' ', header=0, escapechar='\\')
df_ep5 = pd.read_csv('Data/star_wars_epV.txt', sep =' ', header=0, escapechar='\\')
df_ep6 = pd.read_csv('Data/star_wars_epVI.txt', sep =' ', header=0, escapechar='\\')

In [9]:
df_ep4

,character,dialogue
1,THREEPIO,Did you hear that? They've shut down the main...
2,THREEPIO,We're doomed!
3,THREEPIO,There'll be no escape for the Princess this time.
4,THREEPIO,What's that?
5,THREEPIO,I should have known better than to trust the l...
...,...,...
1006,LUKE,"Oh, no!"
1007,THREEPIO,"Oh, my! Artoo! Can you hear me? Say somethi..."
1008,TECHNICIAN,We'll get to work on him right away.
1009,THREEPIO,"You must repair him! Sir, if any of my circui..."


In [10]:
df_ep5

,character,dialogue
1,LUKE,"Echo Three to Echo Seven. Han, old buddy, do y..."
2,HAN,"Loud and clear, kid. What's up?"
3,LUKE,"Well, I finished my circle. I don't pick up an..."
4,HAN,There isn't enough life on this ice cube to fi...
5,LUKE,Right. I'll see you shortly. There's a meteori...
...,...,...
835,LUKE,I'll meet you at the rendezvous point on Tato...
836,LANDO,"Princess, we'll find Han. I promise."
837,LUKE,"Chewie, I'll be waiting for your signal."
838,LUKE,"Take care, you two. May the Force be with you."


In [11]:
df_ep6

,character,dialogue
1,SHUTTLE CAPTAIN,"Command station, this is ST 321. Code Clearanc..."
2,DEATH STAR CONTROLLER,The security deflector shield will be deactiva...
3,SHUTTLE CAPTAIN,We're starting our approach.
4,OFFICER,Inform the commander that Lord Vader's shuttle...
5,OPERATOR,"Yes, sir."
...,...,...
670,LANDO,"Wedge, I don't think we're going to make it."
671,WEDGE,You'll make it. Just follow me Gold Leader.
672,LANDO,I promised to return his ship without a scratc...
673,HAN,Lando...


In [12]:
characters_ep4 = df_ep4.character.unique()
characters_ep5 = df_ep5.character.unique()
characters_ep6 = df_ep6.character.unique()
#print_arr(characters_ep4)
#print_arr(characters_ep5)
#print_arr(characters_ep6)

In [13]:
# all characters
all_characters_arr = np.unique(np.append(np.append(characters_ep4, characters_ep5), characters_ep6))

print_header("All characters")
print_arr(all_characters_arr)

--------------------
-- All characters --
--------------------
size = 129
['ACKBAR' 'ANAKIN' 'ANNOUNCER' 'ASSISTANT OFFICER' 'ASTRO-OFFICER'
 'AUNT BERU' 'BARTENDER' 'BASE VOICE' 'BEN' 'BERU' 'BIB' 'BIGGS'
 'BOBA FETT' 'BOUSHH' 'BUNKER COMMANDER' 'CAMIE' 'CAPTAIN' 'CHIEF'
 'CHIEF PILOT' 'COMMANDER' 'COMMUNICATIONS OFFICER' 'CONTROL OFFICER'
 'CONTROL ROOM COMMANDER' 'CONTROLLER' 'CREATURE' 'DACK' 'DEAK'
 'DEATH STAR CONTROLLER' 'DEATH STAR INTERCOM VOICE' 'DECK OFFICER'
 'DERLIN' 'DODONNA' 'EMPEROR' 'FIRST CONTROLLER' 'FIRST OFFICER'
 'FIRST TROOPER' 'FIXER' 'GANTRY OFFICER' 'GENERAL MADINE' 'GOLD FIVE'
 'GOLD LEADER' 'GOLD TWO' 'GRAY LEADER' 'GREEDO' 'GREEN LEADER' 'GUARD'
 'HAN' 'HAN and LUKE' 'HAN/PILOT' 'HEAD CONTROLLER' 'HOBBIE' 'HUMAN'
 'IMPERIAL OFFICER' 'IMPERIAL SOLDIER' 'INTERCOM VOICE' 'JABBA' 'JANSON'
 'JERJERROD' 'LANDO' 'LEIA' 'LIEUTENANT' 'LUKE' 'LURE' 'MAN'
 'MASSASSI INTERCOM VOICE' 'MEDICAL DROID' 'MON MOTHMA' 'MOTTI'
 'NAVIGATOR' 'NEEDA' 'NINEDENINE' 'OFFICER' 'OFFIC

In [14]:
# characters in every episode
all_trainable_characters_arr = np.array([])

for character in characters_ep4:
    if character in characters_ep5 and character in characters_ep6:
        all_trainable_characters_arr = np.append(all_trainable_characters_arr, character)

print_header("All recurring characters (that are in every episode)")
print_arr(all_trainable_characters_arr)

----------------------------------------------------------
-- All recurring characters (that are in every episode) --
----------------------------------------------------------
size = 8
['THREEPIO' 'LUKE' 'VADER' 'LEIA' 'BEN' 'HAN' 'OFFICER' 'WEDGE']


In [15]:
# check how many times a recurring character says something
trainable_character_counts = (
    df_ep4.character.value_counts()[all_trainable_characters_arr]
    + df_ep5.character.value_counts()[all_trainable_characters_arr]
    + df_ep6.character.value_counts()[all_trainable_characters_arr])

trainable_character_counts = trainable_character_counts.sort_values(ascending=False)

print_header("Recurring characters (sorted by most dialogue)")
print_arr(trainable_character_counts)

# narrow down trainable characters
trainable_character_counts = trainable_character_counts[:6]

print("")
print_header("Trainable characters (sorted by most dialogue)")
print_arr(trainable_character_counts)

----------------------------------------------------
-- Recurring characters (sorted by most dialogue) --
----------------------------------------------------
size = 8
LUKE        494
HAN         459
THREEPIO    301
LEIA        227
VADER       140
BEN         115
WEDGE        33
OFFICER      13
Name: character, dtype: int64

----------------------------------------------------
-- Trainable characters (sorted by most dialogue) --
----------------------------------------------------
size = 6
LUKE        494
HAN         459
THREEPIO    301
LEIA        227
VADER       140
BEN         115
Name: character, dtype: int64


In [16]:
# choose which character the chatbot will train to mimic
CHARACTER_NAME = 'HAN' # CHARACTER_NAME = character to train

print("")
print("! " + "CHARACTER TO TRAIN = " + CHARACTER_NAME + " !")
print("")

print_header(CHARACTER_NAME + "'s lines")
print("Episode IV:")
print_arr(df_ep4.loc[df_ep4.character == CHARACTER_NAME])
print("Episode V:")
print_arr(df_ep5.loc[df_ep5.character == CHARACTER_NAME])
print("Episode VI:")
print_arr(df_ep6.loc[df_ep6.character == CHARACTER_NAME])


! CHARACTER TO TRAIN = HAN !

-----------------
-- HAN's lines --
-----------------
Episode IV:
size = 153
     character                                           dialogue
342        HAN  Han Solo.  I'm captain of the Millennium Falco...
344        HAN  Fast ship?  You've never heard of the Millenni...
346        HAN  It's the ship that made the Kessel run in less...
347        HAN  I've outrun Imperial starships, not the local ...
349        HAN           What is it?  Some kind of local trouble?
...        ...                                                ...
995        HAN                             You're all clear, kid.
996        HAN             Now let's blow this thing and go home!
999        HAN       Great shot, kid.  That was one in a million.
1002       HAN                                         Hey!  Hey!
1004       HAN  Well, I wasn't gonna let you get all the credi...

[153 rows x 2 columns]
Episode V:
size = 182
    character                                         

In [17]:
################
# Prepare data #
################

In [18]:
def create_contextual_dataframe(df):
    context_arr = []
    n = 7 # context window size
    
    # loop over all CHARACTER_NAME's lines and add the previous dialogue as context
    for i in df.loc[df.character == CHARACTER_NAME].index:
      if i < n:
        continue
      row = []
      prev = i - 1 - n # -1 => 6 context + 1 response
      for j in range(i, prev, -1):
        row.append(df.dialogue[j])
      context_arr.append(row)
    
    columns = ['response', 'context'] 
    columns = columns + ['context/' + str(i) for i in range(n - 1)]
    
    return pd.DataFrame.from_records(context_arr, columns=columns)

In [19]:
# make dataframe
df_ep4_contextual = create_contextual_dataframe(df_ep4)
df_ep5_contextual = create_contextual_dataframe(df_ep5)
df_ep6_contextual = create_contextual_dataframe(df_ep6)

print(df_ep4_contextual.shape)
print(df_ep5_contextual.shape)
print(df_ep6_contextual.shape)

df = pd.concat([df_ep4_contextual, df_ep5_contextual, df_ep6_contextual])

print("expected total length: ", df_ep4_contextual.shape[0] + df_ep5_contextual.shape[0] + df_ep6_contextual.shape[0])
print("final shape: ", df.shape)
df.head(2)

(153, 8)
(180, 8)
(124, 8)
expected total length:  457
final shape:  (457, 8)


,response,context,context/0,context/1,context/2,context/3,context/4,context/5
0,Han Solo. I'm captain of the Millennium Falco...,I don't like the look of this.,This is Chewbacca. He's first-mate on a ship ...,No blasters! No blaster!,This little one isn't worth the effort. Come ...,You'll be dead.,I'll be careful than.,Don't insult us. You just watch yourself. We...
1,Fast ship? You've never heard of the Millenni...,"Yes, indeed. If it's a fast ship.",Han Solo. I'm captain of the Millennium Falco...,I don't like the look of this.,This is Chewbacca. He's first-mate on a ship ...,No blasters! No blaster!,This little one isn't worth the effort. Come ...,You'll be dead.


In [20]:
trn_df, val_df = train_test_split(df, test_size=0.1, random_state=0)
trn_df.head(2)

,response,context,context/0,context/1,context/2,context/3,context/4,context/5
94,"No, I need you to talk to the Falcon, find out...","Sir, I'm almost afraid to ask, but...does that...",I'm going to shut down everything but the emer...,"Yes, lord.","Asteroids do not concern me, Admiral. I want ...","Our ships have sighted the Millennium Falcon, ...","Yes, Admiral?","If you're saying coming here was a bad idea, I..."
18,Why do you take this apart now? I'm trying to ...,"Oh, switch off.",Don't try to blame me. I didn't ask you to tur...,I can arrange that. You could use a good kiss!,I'd just as soon kiss a Wookiee.,Am I? Then why are you following me? Afraid I ...,You're imagining things.,No! That's not it. Come on. Aahhh -- uh huh! C...


In [21]:
############
# Training #
############

In [22]:
import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler

from tqdm.notebook import tqdm, trange

from pathlib import Path

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)


try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

# Configs
logger = logging.getLogger(__name__)

MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [35]:
class Args():
    def __init__(self):
        self.output_dir = 'output-HAN'
        self.model_type = 'gpt2'
        self.model_name_or_path = 'microsoft/DialoGPT-small'
        self.config_name = 'microsoft/DialoGPT-small'
        self.tokenizer_name = 'microsoft/DialoGPT-small'
        self.cache_dir = 'cached'
        self.block_size = 512
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 4
        self.per_gpu_eval_batch_size = 4
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 3 # recommended 1 to 2 epochs
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_steps = 3500
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.should_continue = False
        self.seed = 42
        self.local_rank = -1
        self.fp16 = False
        self.fp16_opt_level = 'O1'

args = Args()

In [24]:
def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

class ConversationDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):

        block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)

        directory = args.cache_dir
        cached_features_file = os.path.join(
            directory, args.model_type + "_cached_lm_" + str(block_size)
        )

        if os.path.exists(cached_features_file) and not args.overwrite_cache:
            logger.info("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Creating features from dataset file at %s", directory)

            self.examples = []
            for _, row in df.iterrows():
                conv = construct_conv(row, tokenizer)
                self.examples.append(conv)

            logger.info("Saving features into cached file %s", cached_features_file)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)

In [25]:
# Caching and storing of data/checkpoints

def load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False):
    return ConversationDataset(tokenizer, args, df_val if evaluate else df_trn)


def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
    ordering_and_checkpoint_path = []

    glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))

    for path in glob_checkpoints:
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))

    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    return checkpoints_sorted


def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
    if not args.save_total_limit:
        return
    if args.save_total_limit <= 0:
        return

    # Check if we should delete older checkpoint(s)
    checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
    if len(checkpoints_sorted) <= args.save_total_limit:
        return

    number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
        shutil.rmtree(checkpoint)

In [26]:
def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))
    # add_special_tokens_(model, tokenizer)


    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

# Evaluation of some model

def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

In [27]:
# Main runner

def main(df_trn, df_val):
    args = Args()
    
    if args.should_continue:
        sorted_checkpoints = _sorted_checkpoints(args)
        if len(sorted_checkpoints) == 0:
            raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
        else:
            args.model_name_or_path = sorted_checkpoints[-1]

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup CUDA, GPU & distributed training
    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)

    config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)
    model = AutoModelWithLMHead.from_pretrained(
        args.model_name_or_path,
        from_tf=False,
        config=config,
        cache_dir=args.cache_dir,
    )
    model.to(args.device)
    
    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False)

        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
    if args.do_train:
        # Create output directory if needed
        os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelWithLMHead.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelWithLMHead.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, df_trn, df_val, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results

In [29]:
# small (3 epochs)

In [31]:
main(trn_df, val_df)

01/29/2023 03:45:35 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
01/29/2023 03:45:38 - INFO - __main__ -   Training/evaluation parameters <__main__.Args object at 0x000001D635BA3A30>
01/29/2023 03:45:38 - INFO - __main__ -   Creating features from dataset file at cached
01/29/2023 03:45:38 - INFO - __main__ -   Saving features into cached file cached\gpt2_cached_lm_512
C:\Users\Warre\anaconda3\envs\deep_learning\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
01/29/2023 03:45:38 - INFO - __main__ -   ***** Running training *****
01/29/2023 03:45:38 - INFO - __main__ -     Num examples = 411
01/29/2023 03:45:38 - INFO - __main__ -     Num Epochs = 3
01/29/2023 03:45:38 - INFO - 

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/102 [00:00<?, ?it/s]

Iteration:   0%|          | 0/102 [00:00<?, ?it/s]

Iteration:   0%|          | 0/102 [00:00<?, ?it/s]

01/29/2023 05:37:09 - INFO - __main__ -    global_step = 306, average loss = 2.5049088577819028
01/29/2023 05:37:09 - INFO - __main__ -   Saving model checkpoint to output-small
01/29/2023 05:37:19 - INFO - __main__ -   Evaluate the following checkpoints: ['output-small']
01/29/2023 05:37:20 - INFO - __main__ -   Creating features from dataset file at cached
01/29/2023 05:37:20 - INFO - __main__ -   Saving features into cached file cached\gpt2_cached_lm_512
01/29/2023 05:37:20 - INFO - __main__ -   ***** Running evaluation  *****
01/29/2023 05:37:20 - INFO - __main__ -     Num examples = 46
01/29/2023 05:37:20 - INFO - __main__ -     Batch size = 4


Evaluating:   0%|          | 0/11 [00:00<?, ?it/s]

01/29/2023 05:37:56 - INFO - __main__ -   ***** Eval results  *****
01/29/2023 05:37:56 - INFO - __main__ -     perplexity = tensor(7.4134)


{'perplexity_': tensor(7.4134)}

In [30]:
# small-v2 (2 epochs)

In [27]:
main(trn_df, val_df)

01/29/2023 16:14:03 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
C:\Users\Warre\anaconda3\envs\deep_learning\lib\site-packages\transformers\models\auto\modeling_auto.py:1248: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
01/29/2023 16:14:06 - INFO - __main__ -   Training/evaluation parameters <__main__.Args object at 0x0000021F892A7280>
01/29/2023 16:14:06 - INFO - __main__ -   Creating features from dataset file at cached
01/29/2023 16:14:06 - INFO - __main__ -   Saving features into cached file cached\gpt2_cached_lm_512
C:\Users\Warre\anaconda3\envs\deep_learning\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated an

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/102 [00:00<?, ?it/s]

Iteration:   0%|          | 0/102 [00:00<?, ?it/s]

01/29/2023 16:14:27 - INFO - __main__ -    global_step = 204, average loss = 2.7952919801076255
01/29/2023 16:14:27 - INFO - __main__ -   Saving model checkpoint to output-small-v2
01/29/2023 16:14:30 - INFO - __main__ -   Evaluate the following checkpoints: ['output-small-v2']
01/29/2023 16:14:31 - INFO - __main__ -   Creating features from dataset file at cached
01/29/2023 16:14:31 - INFO - __main__ -   Saving features into cached file cached\gpt2_cached_lm_512
01/29/2023 16:14:31 - INFO - __main__ -   ***** Running evaluation  *****
01/29/2023 16:14:31 - INFO - __main__ -     Num examples = 46
01/29/2023 16:14:31 - INFO - __main__ -     Batch size = 4


Evaluating:   0%|          | 0/11 [00:00<?, ?it/s]

01/29/2023 16:14:31 - INFO - __main__ -   ***** Eval results  *****
01/29/2023 16:14:31 - INFO - __main__ -     perplexity = tensor(9.1171)


{'perplexity_': tensor(9.1171)}

In [31]:
# small-v3 (1 epochs)

In [29]:
main(trn_df, val_df)

01/29/2023 16:18:12 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
01/29/2023 16:18:14 - INFO - __main__ -   Training/evaluation parameters <__main__.Args object at 0x0000021F9A5529D0>
01/29/2023 16:18:14 - INFO - __main__ -   Creating features from dataset file at cached
01/29/2023 16:18:15 - INFO - __main__ -   Saving features into cached file cached\gpt2_cached_lm_512
01/29/2023 16:18:15 - INFO - __main__ -   ***** Running training *****
01/29/2023 16:18:15 - INFO - __main__ -     Num examples = 411
01/29/2023 16:18:15 - INFO - __main__ -     Num Epochs = 1
01/29/2023 16:18:15 - INFO - __main__ -     Instantaneous batch size per GPU = 4
01/29/2023 16:18:15 - INFO - __main__ -     Total train batch size (w. parallel, distributed & accumulation) = 4
01/29/2023 16:18:15 - INFO - __main__ -     Gradient Accumulation steps = 1
01/29/2023 16:18:15 - INFO - __main__ -     Total optimization steps = 102


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/102 [00:00<?, ?it/s]

01/29/2023 16:18:24 - INFO - __main__ -    global_step = 102, average loss = 3.2971490177453733
01/29/2023 16:18:24 - INFO - __main__ -   Saving model checkpoint to output-small-v3
01/29/2023 16:18:27 - INFO - __main__ -   Evaluate the following checkpoints: ['output-small-v3']
01/29/2023 16:18:28 - INFO - __main__ -   Creating features from dataset file at cached
01/29/2023 16:18:28 - INFO - __main__ -   Saving features into cached file cached\gpt2_cached_lm_512
01/29/2023 16:18:28 - INFO - __main__ -   ***** Running evaluation  *****
01/29/2023 16:18:28 - INFO - __main__ -     Num examples = 46
01/29/2023 16:18:28 - INFO - __main__ -     Batch size = 4


Evaluating:   0%|          | 0/11 [00:00<?, ?it/s]

01/29/2023 16:18:29 - INFO - __main__ -   ***** Eval results  *****
01/29/2023 16:18:29 - INFO - __main__ -     perplexity = tensor(12.7401)


{'perplexity_': tensor(12.7401)}